In [1]:
import json
import torch
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import sys
import torch.optim as optim
import json

sys.path.append(os.path.abspath(".."))

#### 1. Download Data

- Go to https://www.nuscenes.org/nuplan

- Sign up or Log in

- Download sub-dataset of choice from nuPlan v1.1 Dataset > nuPlan Train/Val/Test Split

    - In this case, we are using nuPlan Train Split Pittsburgh (28.52 GB)

#### 2. Set Up the NuPlan API

- Make a new environment that meets all of the nuplan requirements

    - conda create -n nuplan python=3.10 -y
    - conda activate nuplan

- Set up all necessary packages

    - pip install "torch==2.3.*" torchvision --index-url https://download.pytorch.org/whl/cu121
        - Note: edit this one depending on CUDA version

    - pip install pyquaternion matplotlib pytest scipy opencv-python requests tqdm numpy pandas shapely pyproj geopandas aioboto3 aiobotocore boto3 retry rasterio omegaconf hydra-core

- Test the cell below to make sure everything is ready

In [ ]:
import sys, importlib, platform
mods = [
    "nuplan",
    "nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_builder",
    "nuplan.planning.scenario_builder.scenario_filter",
    "nuplan.common.actor_state.ego_state",
]
ok = True
for m in mods:
    try:
        importlib.import_module(m)
        print("OK  ", m)
    except Exception as e:
        ok = False
        print("FAIL", m, "->", repr(e))

print("Python:", sys.version.split()[0], "| Platform:", platform.platform())
assert ok, "One or more NuPlan modules failed to import."

#### 3. Process Data into Graphs

- You can change the following parameters:

    - db_dir: the directory that contains the data downloaded from nuPlan

    - out_dir: the directory where you would like to save the processed graphs

    - time_idx: the number of seconds between each frame

    - max_files: the nuPlan dataset is divided into several db files, each containing several 'episodes'. Since there are many files, you can use this parameter to select the maximum number of files that you would like to process. 

In [ ]:
from functions.load_data_NuPlan import load_data

load_data(db_dir='../data/raw/NuPlan/train_pittsburgh',
          time_idx=3,
          max_files=None)

 20%|█▉        | 305/1560 [27:55:41<121:22:31, 348.17s/it]

#### 4. Process Data Tags

- You can change the following parameters:

    - src_dir: the directory that contains the generated json graphs

    - dst_dir: the directory where you would like to save the processed tags

In [2]:
from functions.process_tags_NuPlan import process_tags

process_tags(
    src_dir='../data/graphical/nuplan',
    dst_dir='../data/semantic_tags/NuPlan',
    overwrite=True
)

Done. Wrote: 88370, skipped: 0, errors: 0


#### Quick Visualization of Graph & Tags

In [5]:
from functions.graphs import combined_graph_viewer

ep_num = 5
with open(f"../data/graphical/nuplan/{ep_num}_graph.json", "r") as f:
    graph_data = json.load(f)

cyto = combined_graph_viewer({'graph':graph_data})
display(cyto)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

In [3]:
ep_num = 5
with open(f"../data/semantic_tags/nuplan/{ep_num}_graph.json", "r") as f:
    graph_tags = json.load(f)
graph_tags

{'action_tags': ['stationary'],
 'traffic_control_tags': [],
 'road_feature_tags': [],
 'environment_tags': ['low_visibility_possible', 'night_time']}

In [ ]:
from functions.graphs import graph_to_dfs

nodes_df, edges_df = graph_to_dfs(graph_data)
nodes_df.iloc[8].dropna()

id             vehicle_e_0
type               vehicle
x            587962.181725
y           4475324.279982
z                200.55109
vx                     0.0
vy                     0.0
vz                     0.0
yaw              -0.226276
category           vehicle
tags                    []
Name: 8, dtype: object